In [18]:
import pandas as pd
import matplotlib as plt
import pathlib
import fastai
import pathlib

In [19]:
folder_path = pathlib.Path('./preprocessed/')
filename = '2006-10.csv'
file_path = folder_path.joinpath(filename)
data = pd.read_csv(file_path)

In [20]:
data.head()

,Stock code,market cap,price per sale,current price,price per book,return on assets,return on equity,revenue per share,quarterly revenue growth,total cash per share,...,industry,total cash per dollar,book value per dollar,earnings per dollar,revenue per dollar,free cash flow,price to free cash flow,stock 52 week change,missing,s&p_average_change
0,A,13190000,2.39,32.27,3.55,13.20,34.26,12.408,17.0,5.503,...,Scientific & Technical Instruments,69692161,117411598,86877744,157139802,742001000.0,1.777626e+01,-1.55,0,1.199256
1,AA,24060000,0.84,27.75,1.67,6.14,13.22,32.673,18.9,0.523,...,Aluminum,16338143,516197850,66539664,1020680971,-167999000.0,-1.432151e+02,15.24,0,1.199256
2,AAA,7650000,1.65,56.29,2.85,13.33,23.16,34.300,21.5,5.338,...,Drug Manufacturers - Other,12895030,47879260,10290900,82858660,1000.0,7.650000e+06,1.65,1,1.199256
3,AAC,17470,2.59,0.28,1.27,-1.84,5.84,0.104,193.7,0.031,...,Business Services,6909678,47476178,2228928,23180857,-336000.0,-5.199405e+01,-20.59,2,1.199256
4,AACC,595520,2.32,16.00,2.12,13.73,17.76,6.754,-3.0,1.487,...,Credit Services,3459133,17228207,2791500,15711492,66235000.0,8.991017e+00,-46.34,0,1.199256


In [21]:
data.dtypes

Stock code                   object
market cap                    int64
price per sale              float64
current price               float64
price per book              float64
return on assets            float64
return on equity            float64
revenue per share           float64
quarterly revenue growth    float64
total cash per share        float64
debt equity ratio           float64
book value per share        float64
operating cash flow           int64
volume                        int64
Price per Earnings ratio    float64
earnings per shares         float64
sector                       object
industry                     object
total cash per dollar         int64
book value per dollar         int64
earnings per dollar           int64
revenue per dollar            int64
free cash flow              float64
price to free cash flow     float64
stock 52 week change        float64
missing                       int64
s&p_average_change          float64
dtype: object

In [22]:
numeric_features = data.select_dtypes('number').columns
id_feature = ['Stock_code']
cat_features = data.select_dtypes(exclude = 'number').columns.drop(['Stock_code'])

### Extract target data from the next month

In [23]:
n_filename = '2006-11.csv'
n_file_path = folder_path.joinpath(n_filename)
n_data = pd.read_csv(n_file_path)

In [24]:
n_data.columns

Index(['Stock code', 'market cap', 'price per sale', 'current price',
       'price per book', 'return on assets', 'return on equity',
       'revenue per share', 'quarterly revenue growth', 'total cash per share',
       'debt equity ratio', 'book value per share', 'operating cash flow',
       'volume', 'Price per Earnings ratio', 'earnings per shares', 'sector',
       'industry', 'total cash per dollar', 'book value per dollar',
       'earnings per dollar', 'revenue per dollar', 'free cash flow',
       'price to free cash flow', 'stock 52 week change', 'missing',
       's&p_average_change'],
      dtype='object')

In [25]:
merged_data = pd.merge(left=data,right=n_data.loc[:,['Stock code','current price','s&p_average_change']],how='left',left_on='Stock code',right_on='Stock code')

In [28]:
merged_data['s&p_average_change_y']

0       1.253719
1       1.253719
2       1.253719
3       1.253719
4       1.253719
          ...   
6676    1.241670
6677         NaN
6678    1.241670
6679    1.241670
6680    1.241670
Name: s&p_average_change_y, Length: 6681, dtype: float64

In [33]:
target_var = 'trend_sp'
merged_data['price_change_percentage'] = (merged_data['current_price_y'] - merged_data['current_price_x']) /  merged_data['current_price_x'] *100.
merged_data['trend_sp'] = (merged_data['price_change_percentage'] > merged_data['sp_average_change_y']) & (merged_data['sp_average_change_y']>0)
merged_data = merged_data.drop(['current_price_y','sp_average_change_y','sp_average_change_x','price_change_percentage'],axis=1,errors='ignore')
merged_data.rename({'current_price_x': 'current_price'},axis=1,inplace=True)

merged_data[target_var] = merged_data[target_var].fillna(False)

In [34]:
merged_data[['current_price',target_var]].head()

,current_price,trend_sp
0,32.27,True
1,27.75,True
2,56.29,False
3,0.28,False
4,16.00,True


In [36]:
merged_data[target_var].mean()

0.5470737913486005